# O Uso da Linguagem Julia em Aplicações Paralelas: Desenvolvendo um Sistema de Recomendação

<font color='red'>Anotações de orientação para os autores durante a elaboração do material são postas em vermelho para diferenciar do material do Minicurso. </font>

## <font color='red'> Esqueleto:
 
    1) O que é a Linguagem Julia, e projetos onde é aplicada;
    2) Como instalar, onde fazer o download e como rodar no jupyter notebook, onde encontrar informação sobre Julia.
    3) Sintaxe do Julia: declaração, funções, loops 
    4) Sintaxe: pacotes e macros
    5) Como configurar threads, macros @threads e @spawn e como são usadas 
    6) Atomics e Locks   
    7) Sistema de recomendação com as alterações para multithreading
    8) fim

## O que é a Linguagem Julia ? 

Julia é uma linguagem de programação desenvolvida com a proposta de reconciliar produtividade e desempenho, aliando uma sintaxe próxima de linguagens como Python e um compilador _just-in-time_ , tendo desempenho próximo de linguagens como C.

Julia já tem sido usada em aplicações de larga escala, como: locomoção de robôs em terrenos acidentados (MIT); modelagem macroeconômica, na qual o Julia reduziu o tempo da computação de semanas a dias (Now-Casting Economics); catalogação de objetos astronômicos de forma massivamente paralela (NERSC); sistema de evasão de colisões em pleno voo (Federal Aviation Administration /US).

## Como instalar o Julia e o IJulia

Para instalar o Julia, é recomendável buscar sempre o binário na página inicial. Atualmente o Julia encontra-se na versão 1.5.3 em https://julialang.org/downloads/. O Julia funciona na linha de comando do terminal, muito parecido com o Python, mas é possível utilizá-la com uma IDE. O Juno é a IDE recomendada que funciona sobre o Atom, mais informações em: https://junolab.org/.
![Imgur](https://imgur.com/CB4DkZm.png)

## Como fazer o Julia executar no seu Jupyter Notebook?

<font color='blue'> Adicionar pequeno enxerto sobre o jupyter notebook e sobre o porque escolhemos o jupyter notebook para o ambiente. 
    Adicionar comentário sobre o Pluto. </font>

É possível executar o Julia em seu jupyter notebook, para fazê-lo, instale o pacote IJulia no seu Julia:

In [ ]:
using Pkg; Pkg.add("IJulia"); Pkg.build("IJulia") #não é necessário executar essa linha de código aqui

Este processo informará ao jupyter as especificações necessárias para executar o Julia. Mais informações em: https://julialang.github.io/IJulia.jl/stable/manual/installation/.

Uma vez instalado. Também é possível abrir o jupyter a partir do Julia. Com o I Julia compilado. execute o comando `notebook()`. O jupyter deve abrir o navegador de sua preferência.

In [ ]:
using IJulia; notebook() # este comando compila o pacote IJulia e depois inicia o notebook, 
                         # apenas como exemplo, não é necessário executar

## Metaprogramação

### O que são macros?

Macros são uma forma de utilizar o código da linguagem e transformá-lo para gerar outro pedaço de código. Em outras palavras, é possível ter código como um objeto da própria linguagem. É fundamentalmente diferente de lidar com variáveis e funções, que são expressões que já foram analisados pelo compilador.

In [199]:
x = 20;

In [249]:
x = x+1

23

In [73]:
function soma_1(x)
    x = x + 1
    x
end

soma_1 (generic function with 1 method)

In [252]:
x = soma_1(x)

26

As macros geram e incluem fragmentos de código personalizado durante o tempo de interpretação, portanto, antes da execução do programa completo.

In [ ]:
macro sayhello()
           return :( println("Hello, world!") )
       end

In [264]:
macro soma_unaria(value)
    return :($value + 1)
end

@soma_unaria (macro with 1 method)

In [268]:
x = 20

20

In [270]:
@soma_unaria x

21

In [244]:
@somaunaria 15

21

In [245]:
macro soma_um(value)
    local x = :($value) + 1
    x
end 

@soma_um (macro with 1 method)

In [246]:
@soma_um 15

16

In [53]:
macro soma(args...)
    @assert args[2] == "mais"
    args[1] + args[3]
end

@soma (macro with 1 method)

In [56]:
@soma 2 "mais" 2

4

In [58]:
@soma(2, "mais", 2)

4

### Literais, símbolos e expressões

Símbolos, literais e expressões são strings internadas, que passaram pela análise sintática, mas que ainda não foram avaliadas.

In [114]:
macro whatisit(exp)
    println(typeof(:($exp)))
end

@whatisit (macro with 1 method)

In [115]:
@whatisit(2)

Int64


In [116]:
@whatisit("Hello")

String


In [117]:
@whatisit(x)

Symbol


In [118]:
@whatisit(2 + 2)

Expr


In [120]:
dump(:(2+2))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 2
    3: Int64 2


In [121]:
+(2,2)

4

In [149]:
eval(2+2)

4

**Note:** Se tirarmos os comentários da soma unária, veremos que o que é passado para a macro e o símbolo avaliado são duas coisas distintas. Isso porque o literal, símbolo ou expressão é o que é passado em tempo de compilação. A avaliação deles é que é realizado em tempo de execução.

In [176]:
macro soma_unaria_(value)
    println(typeof(value))
    value  = eval(value)
    println(value)
    value + 1
end 

@soma_unaria_ (macro with 1 method)

In [177]:
@soma_unaria_(x)

Symbol
20


21

### Assim como funções, macros também podem ter multiple dispatch

In [97]:
macro yousaygoodbye()
    :(println("and I say hello"))
end
macro yousaygoodbye(exp)
    :(println("and I say hello, ",$exp))
end

@yousaygoodbye (macro with 2 methods)

In [93]:
@yousaygoodbye

and I say hello


In [98]:
@yousaygoodbye "Julia"

and I say hello, Julia


## Algumas macros interessantes!
Usar macros é uma forma muito limpa e prática de representar um código:

Existem mascros padrões da linguagem e muitas outras mais em pacotes do Julia

adicionar @inbounds, @btime

### @ccall

In [154]:
ccall(:strlen, Cint, (Cstring,), "Maria") #

5

In [155]:
@ccall strlen("Maria"::Cstring)::Cint # macro ccall

5

### @elapsed e @time

In [182]:
@elapsed peakflops()

0.455398732

In [180]:
Base.remove_linenums!(@macroexpand @elapsed peakflops())

quote
    while false
    end
    local var"#148#t0" = Base.time_ns()
    [(j - 1) * 100 + i for i = 1:100, j = 1:100]
    (Base.time_ns() - var"#148#t0") / 1.0e9
end

In [184]:
@time peakflops()

  0.348026 seconds (11 allocations: 61.188 MiB, 1.21% gc time)


4.89565316118536e10

## Adicionar parte sobre pacote

Falar sobre Pkg3 - sistema de gerenciamento de pacotes, dependências, versão etc.

Falar sobre pacotes famosos

## Threads

No Julia, até o presente momento, não é possível alterar a quantidade de threads em tempo de execução.

## Como configurar a quantidade de threads no ambiente?

Iremos trabalhar com threads, para isso, é necessário que o Julia seja inicializado com a quantidade de threads configuradas.

#### Como variável de ambiente
**Ubuntu** :
É possível definir um valor para uma variável de ambiente antes de iniciar o Julia:

<img align='left' src='https://i.imgur.com/fxYMIsT.png'> </img>

Também é possível definir o valor de JULIA_NUM_THREADS no arquivo bash para definir a quantidade de threads de forma permanente.

#### Ao iniciar o Julia no terminal
Para definir threads ao iniciar o Julia, basta passar o número de threads desejado como argumento ao iniciar o julia

<img align='left' src='https://i.imgur.com/AtPZS1V.png'> </img>

#### No Atom: 
Com o Juno instalado, vá na Aba Juno -> Settings -> Julia Options -> Number of Threads. Depois, reinicie o Atom.

<img align='left' src='https://i.imgur.com/vu4f4RU.png'> </img>

In [187]:
using Pkg; Pkg.add("BenchmarkTools");

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [188]:
using BenchmarkTools
using Base.Threads

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1278


## @threads for loops

A macro @threads é a primeira macro que iremos ver. Ela é utilizada para loops e divide a região onde será feita a paralelização das operações entre as threads disponíveis. O modelo atual de @threads em Julia funciona com 

In [190]:
Threads.nthreads()

2

In [196]:
function threadid_array(n)
    arr = Array{Int64}(undef, n)
    @threads for i = 1:length(arr)
        arr[i] = Threads.threadid()
    end
    return arr
end
threadid_array(9)

9-element Array{Int64,1}:
 1
 1
 1
 1
 1
 2
 2
 2
 2